In [1]:
import requests
import re
import bs4
from bs4 import BeautifulSoup

In [2]:
def get_page(url):
    r = requests.get(url)
    r.encoding = 'Windows-1251'
    return r.text

Не обрабатываются 2007, 2006 и 2001 годы. Ссылка на 2007 год на самом деле ведет в 2019, ссылка на 2006 не работает, а по ссылке на 2001 ничего не открывается

In [3]:
# штука которая достает ссылки на архивы разных годов
def get_year_links():
    link_list = []
    archive_adress = 'http://www.gazetayuga.ru/archive.htm'
    soup = BeautifulSoup(get_page(archive_adress))
    links = soup.find('td', {'class':'hc'})
    for line in links.find_all('a'):
        link = 'http://www.gazetayuga.ru/' + line.get("href")
        link_list.append(link)
    return link_list[:12] + link_list[14:-1]

In [4]:
get_year_links()

['http://www.gazetayuga.ru/archive.htm',
 'http://www.gazetayuga.ru/archive2018.htm',
 'http://www.gazetayuga.ru/archive2017.htm',
 'http://www.gazetayuga.ru/archive2016.htm',
 'http://www.gazetayuga.ru/archive2015.htm',
 'http://www.gazetayuga.ru/archive2014.htm',
 'http://www.gazetayuga.ru/archive2013.htm',
 'http://www.gazetayuga.ru/archive2012.htm',
 'http://www.gazetayuga.ru/archive2011.htm',
 'http://www.gazetayuga.ru/archive2010.htm',
 'http://www.gazetayuga.ru/archive2009.htm',
 'http://www.gazetayuga.ru/archive2008.htm',
 'http://www.gazetayuga.ru/archive2005.htm',
 'http://www.gazetayuga.ru/archive2004.htm',
 'http://www.gazetayuga.ru/archive2003.htm',
 'http://www.gazetayuga.ru/archive2002.htm']

In [5]:
# штука которая достает ссылки на разные выпуски одного года
def get_year_publications(year_link):
    link_list = []
    soup = BeautifulSoup(get_page(year_link))
    links = soup.find('td', {'class':'lg'}).pre
    
    for line in links.find_all('a'):
        link = 'http://www.gazetayuga.ru/' + line.get("href")
        link_list.append(link)
        
    return link_list

Ошибки: если не удалось обработать какую-то страницу/статью.
Иногда почему-то в списке публикаций в архиве обрабатывается не только в ссылка на html, но и в ссылка на pdf. Соответственно ссылка на pdf отражается в ошибках.

In [6]:
def get_articles_from_publication(publication_link, file_to_write):
    try:
        url = publication_link 
        soup = BeautifulSoup(get_page(publication_link))
        st = soup.find('td', {'valign':'top', 'align':'center'}).text
        date = re.findall('\d{1,2} \D+ \d{4}', st)[0]

        new_soup = soup.find('table', {'cellspacing':'1', 'cellpadding':'1', 'border':'0', 'width':'100%'}).tbody
        stuff = new_soup.find_all('td', {'class':'lg'})

        
        for thing in stuff:
            for child in thing.children:
                if isinstance(child, bs4.element.Tag) and child.name == 'div' and child.attrs == {'class': ['tit']}:
                    try:
                        title_s = child
                        title = ' '.join(title_s.text.split())
                        if not title:
                            title = 'None'

                        text_s = title_s.next_sibling.next_sibling
                        text = text_s.text
                        if not text:
                            text = 'None'

                        is_author = text_s.next_sibling.next_sibling
                        if is_author.name == 'div' and is_author.attrs == {'class':['tit2']}:
                            author = ' '.join(is_author.text.split())
                        else:
                            author = 'None'

                
                        file_to_write.write('=====')
                        file_to_write.write('\n')
                        file_to_write.write(url)
                        file_to_write.write('\n')
                        file_to_write.write('Газета Юга')
                        file_to_write.write('\n')
                        file_to_write.write(date)
                        file_to_write.write('\n')
                        file_to_write.write(author)
                        file_to_write.write('\n')
                        file_to_write.write(title)
                        file_to_write.write('\n')
                        file_to_write.write(text)
                        file_to_write.write('\n')
                    
                    except:
                        print('Что-то пошло не так:')
                        print(url)
                        continue
                    
    except:
        print('Что-то пошло не так:')
        print(url)
        return

In [7]:
def dump_all_to_file(file):
    f = open(file,"w")
    
    year_links = get_year_links()
    for year_link in year_links:
        year_publications = get_year_publications(year_link)
        for publication in year_publications:
            get_articles_from_publication(publication, f)
            
    f.close()
    return


In [8]:
dump_all_to_file('articles.txt')

Что-то пошло не так:
http://www.gazetayuga.ru/archive/2017/02.htm
Что-то пошло не так:
http://www.gazetayuga.ru/archive/2014/11.htm
Что-то пошло не так:
http://www.gazetayuga.ru/archive/2014/14.htm
Что-то пошло не так:
http://www.gazetayuga.ru/archive/2013/06.htm
Что-то пошло не так:
http://www.gazetayuga.ru/archive/2009/34.htm
Что-то пошло не так:
http://www.gazetayuga.ru/archive/2005/37.htm
Что-то пошло не так:
http://www.gazetayuga.ru/http://www.gazetayuga.ru/pdf/2005/42.pdf
Что-то пошло не так:
http://www.gazetayuga.ru/archive/2005/26.htm
Что-то пошло не так:
http://www.gazetayuga.ru/archive/2005/27.htm
Что-то пошло не так:
http://www.gazetayuga.ru/http://zerkalo.gazetayuga.ru/pdf/2004/01.pdf
Что-то пошло не так:
http://www.gazetayuga.ru/archive/2004/01.htm
Что-то пошло не так:
http://www.gazetayuga.ru/http://zerkalo.gazetayuga.ru/pdf/2004/02.pdf
Что-то пошло не так:
http://www.gazetayuga.ru/http://zerkalo.gazetayuga.ru/pdf/2004/03.pdf
Что-то пошло не так:
http://www.gazetayuga.ru/

Что-то пошло не так:
http://www.gazetayuga.ru/http://zerkalo.gazetayuga.ru/pdf/2002/10.pdf
Что-то пошло не так:
http://www.gazetayuga.ru/http://zerkalo.gazetayuga.ru/pdf/2002/36.pdf
Что-то пошло не так:
http://www.gazetayuga.ru/http://zerkalo.gazetayuga.ru/pdf/2002/11.pdf
Что-то пошло не так:
http://www.gazetayuga.ru/http://zerkalo.gazetayuga.ru/pdf/2002/37.pdf
Что-то пошло не так:
http://www.gazetayuga.ru/http://zerkalo.gazetayuga.ru/pdf/2002/12.pdf
Что-то пошло не так:
http://www.gazetayuga.ru/http://zerkalo.gazetayuga.ru/pdf/2002/38.pdf
Что-то пошло не так:
http://www.gazetayuga.ru/http://zerkalo.gazetayuga.ru/pdf/2002/13.pdf
Что-то пошло не так:
http://www.gazetayuga.ru/http://zerkalo.gazetayuga.ru/pdf/2002/39.pdf
Что-то пошло не так:
http://www.gazetayuga.ru/http://zerkalo.gazetayuga.ru/pdf/2002/14.pdf
Что-то пошло не так:
http://www.gazetayuga.ru/http://zerkalo.gazetayuga.ru/pdf/2002/40.pdf
Что-то пошло не так:
http://www.gazetayuga.ru/http://zerkalo.gazetayuga.ru/pdf/2002/15.pdf